In [1]:
import pandas as pd
import bqplot
import numpy as np
import traitlets
import ipywidgets
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
mySelectedLabel = ipywidgets.Label()

In [3]:
buildings = pd.read_csv('building_inventory.csv',
               na_values={'Square Footage': 0, 
                         'Year Acquired': 0,
                         'Year Constructed': 0, 
                         'Floors':0})

In [4]:
buildings_heatmap_data = pd.pivot_table(buildings,
                                       index = ['Congress Dist'],
                                       values = ['Square Footage'],
                                       columns = ['Agency Name'],
                                       aggfunc = np.sum)

In [5]:
Agency_Name = np.asarray(buildings_heatmap_data.columns.levels[1].to_list())

In [6]:
Congress_Dist = np.asarray(buildings_heatmap_data.index.to_list())

In [7]:
Square_Footage = buildings_heatmap_data.values

In [8]:
#heatmap
col_sc = bqplot.ColorScale(scheme="RdPu", min=np.nanmin(buildings_heatmap_data), max=np.nanmax(buildings_heatmap_data)) 

x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')

x_ax = bqplot.Axis(scale = x_sc, label='Agency Names')
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical', 
                   label = 'Congress Dist')

heat_map = bqplot.GridHeatMap(color = buildings_heatmap_data,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'},
                              anchor_style = {'fill':'blue'}, 
                              selected_style = {'opacity': 1.0},
                              unselected_style = {'opacity': 1.0})

In [9]:
total_year = buildings.groupby("Year Acquired")["Square Footage"].sum()

In [10]:
#line graph
i,j = 17, 6

Name = [Agency_Name[j]] # min/max longitude
Dist = [Congress_Dist[i],Congress_Dist[i+1]] # min/max latitude
mask_name = []

#prepare mask
for i in range(len(buildings['Agency Name'])):
    mask_name.append(Name[0])

name_mask = (buildings['Agency Name'] == mask_name)
region_mask = ( (buildings['Congress Dist'] >= Dist[0]) & (buildings['Congress Dist'] < Dist[1]))

final_mask = (name_mask & region_mask)

#get data attributes satisfy the requirement
mask_buildings = buildings[final_mask]
sorted_mask_buildings = mask_buildings.sort_values('Year Acquired')
agg_b = sorted_mask_buildings.groupby("Year Acquired")["Square Footage"].sum()

#plot line graph
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

lines = bqplot.Lines(x = sorted_mask_buildings['Year Acquired'].unique(), y = agg_b, scales = {'x': x_sc, 'y': y_sc})

ax_x = bqplot.Axis(scale = x_sc, label = 'Year Acquired')
ax_y = bqplot.Axis(scale = y_sc, label = 'Total Square Footage', orientation = 'vertical')

In [11]:
#interaction between line graph and heatmap
def get_data_value(change):
    if len(change['owner'].selected) == 1:
        i, j = change['owner'].selected[0]
        v = buildings_heatmap_data.iloc[i,j]
        mySelectedLabel.value = 'Total Square Footage = ' + str(v)  + ', Agency: ' + Agency_Name[j] + ', Congress District: ' + str(i)
        
        #line graph
        Name = [Agency_Name[j]] 
        if i == Congress_Dist[-1]:
            Dist = [Congress_Dist[i],Congress_Dist[i]+1]
        else:
            Dist = [Congress_Dist[i],Congress_Dist[i+1]]
        mask_name = []

        #prepare mask
        for i in range(len(buildings['Agency Name'])):
            mask_name.append(Name[0])

        name_mask = (buildings['Agency Name'] == mask_name)
        region_mask = ( (buildings['Congress Dist'] >= Dist[0]) & (buildings['Congress Dist'] < Dist[1]))
        
        final_mask = (name_mask & region_mask)
        
        #get data attributes satisfy the requirement
        mask_buildings = buildings[final_mask]
        sorted_mask_buildings = mask_buildings.sort_values('Year Acquired')
        agg_b = sorted_mask_buildings.groupby("Year Acquired")["Square Footage"].sum()
        
        
        lines.x = sorted_mask_buildings['Year Acquired'].unique()
        lines.y = agg_b

heat_map.observe(get_data_value, 'selected')

In [12]:
fig_heatmap = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax])
fig_line = bqplot.Figure(marks = [lines], axes = [ax_x, ax_y])

Each square represent one single congress district and agency name. Example, between congress dist 17 and 18, the line plot will only plot when it equals to 17.

In [18]:
fig_heatmap.layout.min_width='500px'
fig_line.layout.min_width='500px'

myDashboard = ipywidgets.VBox([mySelectedLabel, ipywidgets.HBox([fig_heatmap,fig_line])])
myDashboard

End of homework 6

Some extra stuff not in the homework, the total square footage vs the year

In [14]:
total_year = buildings.groupby("Year Acquired")["Square Footage"].sum()

In [19]:
x_scale = bqplot.LinearScale()
y_scale = bqplot.LinearScale()

line_graph = bqplot.Lines(x = buildings['Year Acquired'].sort_values(ascending = True).unique(), y = total_year, scales = {'x': x_scale, 'y': y_scale})

x_axis = bqplot.Axis(scale = x_scale, label = 'Year Acquired')
y_axis = bqplot.Axis(scale = y_scale, label = 'Total Square Footage that year', orientation = 'vertical')

graph = bqplot.Figure(marks = [line_graph], axes = [x_axis, y_axis])
graph

Figure(axes=[Axis(label='Year Acquired', scale=LinearScale()), Axis(label='Total Square Footage that year', or…